In [0]:
%sql
USE CATALOG hive_metastore

In [0]:
%sql
CREATE TABLE employees(
  id INT,
  name STRING,
  salary DOUBLE 
)

In [0]:
%sql
INSERT INTO employees 
VALUES 
(1, 'John', 100000.00),
(2, 'Josh', 200000.00);

INSERT INTO employees 
VALUES 
(3, 'Jane', 500);

INSERT INTO employees 
VALUES 
(4, 'Johnes', 123.00),
(5, 'Joshes', 456.00);

INSERT INTO employees 
VALUES 
(6, 'Rido', 3450.00),
(7, 'Sati', 3235.00);

In [0]:
%sql
SELECT * FROM employees;

In [0]:
%sql
DESCRIBE DETAIL employees;

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/employees'

In [0]:
%sql
UPDATE employees
SET salary = 36.18
WHERE id = 1;

In [0]:
%sql
DESCRIBE HISTORY employees;

In [0]:
%fs ls 'dbfs:/user/hive/warehouse/employees/_delta_log'

In [0]:
%fs head 'dbfs:/user/hive/warehouse/employees/_delta_log/00000000000000000001.00000000000000000006.compacted.json'

### TIME TRAVEL trong DELTA LAKE

Có hai cách thường sử dụng nhằm truy vấn các phiên bản cũ của dữ liệu:

- **Sử dụng mốc thời gian (timestamp):**

```sql
  SELECT * FROM my_table TIMESTAMP AS OF "2025-18-12 16:22:36 UTC"
```  

- **Sử dụng version number:**
```sql
  SELECT * FROM my_tables VERSION AS OF 36
  SELECT * FROM my_tables@v36
```  

---

#### ROLLBACK VERSIONS: Hỗ trợ khôi phục dữ liệu

```sql
ROLLBACK
RESTORE TABLES
```

---

#### Compaction: Gom các file nhỏ thành file lớn để tối ưu hóa cho tác vụ READ

Các tác vụ hàng ngày của Delta Lake nhằm đảm bảo tính ACID cho các transactions, do đó mỗi job chạy sẽ sinh ra ít nhất hai file: một file parquet và một file transaction log.

- micro-batch nhỏ → sinh ra nhiều file nhỏ
- insert/delete/update: Mỗi thao tác là riêng biệt, độc lập (isolation) nên sẽ sinh ra nhiều file nhỏ

Nhiều file nhỏ → Spark phải mở nhiều file, scan nhiều metadata khác nhau → Gây tổn hao tài nguyên không cần thiết.

> **==> Compaction được sinh ra để giải quyết vấn đề này:**  
> Nó gộp các file nhỏ thành file lớn hơn, giúp Spark giảm số lượng file cần mở và scan metadata, từ đó tiết kiệm tài nguyên, chi phí và tăng hiệu năng đọc dữ liệu.

#### INDEXING
- Z-order indexing: 